In [1]:
import pandas as pd

In [2]:
file_path = "역정보+역행정동정보.xlsx"  # 원본 엑셀 파일 경로
output_file_path = "인코딩 완료_역정보_역행정동정보.xlsx"  # 저장할 파일 경로

In [3]:
# 모든 시트 로드
xls = pd.ExcelFile(file_path)
sheets = {sheet: xls.parse(sheet) for sheet in xls.sheet_names}  # 모든 시트 읽기

In [4]:
date_station_map = {
    "2024-04": ["양촌", "구래", "마산", "장기", "운양", "걸포북변", "사우", "풍무", "고촌", "김포공항"],
    "2024-08": ["별내별가람", "오남", "진접", "별내", "다산", "장자호수공원", "구리", "동구릉"],
    "2024-09": ["인천공항제1터미널", "인천공항제2터미널"],
    "2024-11": ["지축", "삼송", "원흥", "원당", "화정", "대곡", "백석", "마두", "정발산", "주엽", "대화",
                "한국항공대", "강매", "행신", "능곡", "대곡", "곡산", "백마", "풍산", "일산", "탄현",
                "능곡", "대곡", "곡산", "백마", "풍산", "일산",
                "선바위", "경마공원", "대공원", "과천", "정부과천청사"]
}

In [6]:
# 모든 시트에 대해 적용
for sheet_name, df in sheets.items():
    # 🔹 "2024_1월" 등 원본 열을 복사하여 "2024_1월_copy" 형태의 새 열 생성
    for col in df.columns:
        if "2024년" in col and "_copy" not in col:  # 원본 열만 선택
            df[col + "_copy"] = df[col]  # "_copy" 버전 추가

    # 🔹 "2024_"와 "copy"가 포함된 열만 선택
    date_columns = [col for col in df.columns if "2024년" in col and "copy" in col]

    # 🔹 각 날짜별 end_station 조건 적용
    for col in date_columns:
        year_month = col.split("_")[0].replace("년 ", "-").replace("월", "")  # "2024-04" 같은 형식으로 변환

        for date, stations in date_station_map.items():
            if year_month >= date:  # 해당 날짜 이후부터 1로 설정
                df[col] = df.apply(lambda row: 1 if row.get("end_station") in stations else row[col], axis=1)
            else:  # 지정 날짜 이전은 0으로 설정
                df[col] = df.apply(lambda row: 0 if row.get("end_station") in stations else row[col], axis=1)

    # 🔹 추가 기능: date_station_map에 없는 end_station 처리
    for col in date_columns:
        df[col] = df.apply(
            lambda row: 1 if row.get("end_station") not in sum(date_station_map.values(), []) and row.get("climate_pass") == "O" else row[col],
            axis=1
        )

        df[col] = df.apply(
            lambda row: 0 if row.get("end_station") not in sum(date_station_map.values(), []) and row.get("climate_pass") == "X" else row[col],
            axis=1
        )

# 변경된 데이터 저장
with pd.ExcelWriter(output_file_path) as writer:
    for sheet_name, df in sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"업데이트된 파일이 저장되었습니다: {output_file_path}")

업데이트된 파일이 저장되었습니다: 인코딩 완료_역정보_역행정동정보.xlsx
